In [ ]:
#!/usr/bin/env python
# coding: utf-8

from aopy import datareader, datafilter
from ecog_is2s import EcogDataloader, Training
from ecog_is2s.model import Encoder, Decoder, Seq2Seq
from ecog_is2s.model import Util

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import SequentialSampler, BatchSampler, SubsetRandomSampler
from torch.utils.data import TensorDataset, random_split

import spacy
import numpy as np
import pandas as pd
# import sklearn
import scipy as sp

import random
import math
import time

# import progressbar as pb
from datetime import datetime
import os
import sys

import matplotlib.pyplot as plt

import pickle as pkl


In [ ]:
# seed RNG for pytorch/np
SEED = 5050
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# set device - CUDA if you've got it
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('mounting to device: {}'.format(device))


In [ ]:
# load data
platform_name = sys.platform
if platform_name == 'darwin':
    # local machine
    data_file_full_path = '/Volumes/Samsung_T5/aoLab/Data/WirelessData/Goose_Multiscale_M1/180325/001/rec001.LM1_ECOG_3.clfp.dat'
    mask_file_path = "/Volumes/Samsung_T5/aoLab/Data/WirelessData/Goose_Multiscale_M1/180325/001/rec001.LM1_ECOG_3.clfp.mask.pkl"
    model_save_dir_path = '/Volumes/Samsung_T5/aoLab/Data/models/pyt/seq2seq/'
elif platform_name == 'linux2':
    # HYAK, baby!
    data_file_full_path = '/gscratch/stf/manolan/Data/WirelessData/Goose_Multiscale_M1/180325/001/rec001.LM1_ECOG_3.clfp.dat'
    mask_file_path = "/gscratch/stf/manolan/Data/WirelessData/Goose_Multiscale_M1/180325/001/rec001.LM1_ECOG_3.clfp.mask.pkl"
elif platform_name == 'linux':
    # google cloud, don't fail me now
    data_file_full_path = '/home/mickey/rec001.LM1_ECOG_3.clfp.dat'
    mask_file_path = '/home/mickey/rec001.LM1_ECOG_3.clfp.mask.pkl'

data_in, data_param, data_mask = datareader.load_ecog_clfp_data(data_file_name=data_file_full_path)
srate_in= data_param['srate']
num_ch = data_param['num_ch']
# we already found the appropriate data masks, so just load them in
with open(mask_file_path, 'rb') as f:
    mask_data = pkl.load(f)
hf_mask = mask_data["hf"]
sat_mask = mask_data["sat"]


In [ ]:
# mask data array, remove obvious outliers
data_in[:,np.logical_or(hf_mask,sat_mask)] = 0.

# downsample data
srate_down = 250

total_len_T = 1*60 # I just don't have that much time!
total_len_n = total_len_T*srate_in
data_idx = data_in.shape[1]//2 + np.arange(total_len_n)
print('Downsampling data from {0} to {1}'.format(srate_in,srate_down))
data_in = np.float32(sp.signal.decimate(data_in[:,data_idx],srate_in//srate_down,axis=-1))


In [ ]:
# create dataset object from file
srate = srate_in
# data_in = np.double(data_in[:,:120*srate])
enc_len = 10
dec_len = 1
seq_len = enc_len+dec_len # use ten time points to predict the next time point

data_tensor = torch.from_numpy(sp.stats.zscore(data_in.view().transpose()))
if device == 'cuda:0':
    data_tensor.cuda()
print(data_tensor.size)
dataset = EcogDataloader.EcogDataset(data_tensor,device,seq_len) ## make my own Dataset class
idx_all = np.arange(dataset.data.shape[0]) # should these be packed into the dataset class
sample_idx = idx_all[:-seq_len]


In [ ]:
# build the model, initialize
INPUT_SEQ_LEN = enc_len
OUTPUT_SEQ_LEN = dec_len # predict one output state from 10 inputs prior
INPUT_DIM = num_ch
OUTPUT_DIM = num_ch
HID_DIM = num_ch
N_ENC_LAYERS = 1 
N_DEC_LAYERS = 1
ENC_DROPOUT = np.float32(0.5)
DEC_DROPOUT = np.float32(0.5)

enc = Encoder.Encoder_GRU(INPUT_DIM, HID_DIM, N_ENC_LAYERS, ENC_DROPOUT)
dec = Decoder.Decoder_GRU(OUTPUT_DIM, HID_DIM, N_DEC_LAYERS, DEC_DROPOUT)

model = Seq2Seq.Seq2Seq_GRU(enc, dec, device).to(device)
model.apply(Util.init_weights)

print(f'The model has {Util.count_parameters(model):,} trainable parameters')

optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss()


In [ ]:
# take a look at the model state dict
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
# make a function to save the current network state
def save_state(model,epoch,optimizer,loss,data_path,PATH):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'data_path': data_path,
            }, PATH)

In [ ]:
# create training session directory
time_str = datetime.strftime(datetime.now(),'%Y%m%d%H%M%S') # I may do well to pack this into util
model_save_path = os.path.join(model_save_dir_path,'enc{}_dec{}_nl{}_{}.pt'.format(enc_len,dec_len,N_ENC_LAYERS,time_str))
print(model_save_path,os.path.exists(model_save_dir_path))

In [ ]:
# save the initial model state
save_state(model,0,optimizer,np.inf,'./test_save.pt')
asdf = torch.load('test_save.pt')
# print(asdf)
model.load_state_dict(asdf['model_state_dict']) # :)

In [ ]:
# get to training!
train_frac = 0.8
test_frac = 0.2
valid_frac = 0.0
BATCH_SIZE = 1
N_EPOCHS = 1 # please clap
CLIP = 1

best_test_loss = float('inf')

train_loss = np.zeros(N_EPOCHS)
train_batch_loss = []
test_loss = np.zeros(N_EPOCHS)
test_batch_loss = []

# make figure (and a place to save it)
f = plt.figure()
ax = f.add_subplot(1,1,1)

for e_idx, epoch in enumerate(range(N_EPOCHS)):
    
    start_time = time.time()
    
    # get new train/test splits
    train_loader, test_loader, _ = EcogDataloader.genLoaders(dataset, sample_idx, train_frac, test_frac, valid_frac, BATCH_SIZE)
    
    print('Training Network:')
    train_loss[e_idx], trbl_ = Training.train(model, train_loader, optimizer, criterion, CLIP)
    train_batch_loss.append(trbl_)
    print('Testing Network:')
    test_loss[e_idx], tebl_ = Training.evaluate(model, test_loader, criterion)
    test_batch_loss.append(tebl_)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = Util.epoch_time(start_time, end_time)
    
    if test_loss[e_idx] < best_test_loss:
        best_test_loss = test_loss[e_idx]
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss[e_idx]:.3g}')
    print(f'\t Test Loss: {test_loss[e_idx]:.3g}')
    
    ax.plot(e_idx,train_loss[e_idx],'b.')
    ax.plot(e_idx,test_loss[e_idx],'r.')
    
    # print the figure; continuously overwrite (like a fun stock ticker)
    f.savefig('training_progress.png')

# save model
save_state(model,e_idx,optimizer,np.inf,model_save_path)